In [1]:
# default_exp utils

# Utils

> help functions

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
from dataer_tool.imports import *

In [4]:
#export
def get_count_data_datetime(input_df, count_column, date_time_column:str=None, agg_method='sum', cumsum:bool=False, resample_mode:str='d', extra_meta_column:str=None):
    date_df = input_df
    extra_meta_keys = list(date_df[extra_meta_column].value_counts().index) if extra_meta_column is not None else None
    if date_time_column is not None:
        if 'int' in str(date_df[date_time_column].dtype).lower(): date_df[date_time_column] = date_df[date_time_column].astype("string")
        date_df[date_time_column] = pd.to_datetime(date_df[date_time_column])
        date_df = date_df.set_index(date_time_column)
    if date_time_column is None: date_time_column = date_df.index.name = "_index_column_title"
    if resample_mode in ['d', 'w', 'm', 'y']:
        date_df_iter = date_df.resample(resample_mode)
    else:
        date_df_iter = date_df.groupby(date_time_column)
    agg_row = date_df_iter[count_column].agg(agg_method)
    if cumsum: agg_row = agg_row.cumsum()
    count_data = agg_row.to_dict()
    if extra_meta_column is not None:
        extra_output_data = {k: [] for k in extra_meta_keys}
        for o in date_df_iter:
            extra_data_item = o[1].groupby(extra_meta_column)[count_column].sum().to_dict()
            for extra_meta_key in extra_meta_keys:
                extra_output_data[extra_meta_key].append(extra_data_item.get(extra_meta_key, 0))
    else:
        extra_output_data = None
    return (count_data, extra_output_data) if extra_output_data is not None else count_data

In [5]:
df = pd.DataFrame({
 'date': {0: '2000-03-01',
          1: '2000-03-02',
          2: '2000-03-03',
          3: '2000-03-06',
          4: '2000-03-07',
          5: '2000-03-08',
          6: '2000-03-09',
          7: '2000-03-10'},
 'open': {0: 118.56,
          1: 127.0,
          2: 124.87,
          3: 126.0,
          4: 126.44,
          5: 122.87,
          6: 120.87,
          7: 121.69},
 'cat': {0: 'A',
         1: 'B',
         2: 'C',
         3: 'B',
         4: 'A',
         5: 'A',
         6: 'C',
         7: 'C'}
  })

In [6]:
from dataer_tool.visualize.bokeh import line_figure_datetime
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [7]:
data = get_count_data_datetime(df, count_column='open', date_time_column='date')

p = line_figure_datetime(data, title="趋势图", draw_circle=True)
show(p)

/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.background_fill_alpha` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.location` on

In [8]:
data = get_count_data_datetime(df, count_column='open', date_time_column='date', extra_meta_column='cat')

p = line_figure_datetime(data[0], title="趋势图", draw_circle=True, tooltips_metadata=data[1])
show(p)

/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.background_fill_alpha` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.location` on

In [9]:
data = get_count_data_datetime(df, count_column='open', date_time_column='date', extra_meta_column='cat', cumsum=True)

p = line_figure_datetime(data[0], title="趋势图", draw_circle=True, tooltips_metadata=data[1])
show(p)

/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.background_fill_alpha` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/media/storage_0x00/ENV/miniconda3/envs/dataer_tool/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.location` on

In [10]:
#export
def fmt_str_in_file(file_path, **kwargs):
    with open(file_path, 'r') as f:
        data = f.read()
    return data.format(**kwargs)

In [12]:
! echo "{foo} ... {bar}" > foo.bar
print(fmt_str_in_file('foo.bar', foo='123', bar='abc'))
! rm foo.bar


123 ... abc

